# **Random Forest**
- Using data that was not annotated
- Training it on the unbalanced data
- Using avgpool
- 6 second window

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
# Standard libraries
import numpy as np
import pandas as pd
import time
import os

# For audio
from IPython.display import Audio
import librosa

# For preprocessing
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import OneHotEncoder
import tensorflow as tf

# For modeling
from sklearn.metrics import classification_report, roc_auc_score, f1_score
from sklearn.neighbors import KNeighborsClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score

# Operational
from tqdm import tqdm
import matplotlib.pyplot as plt
import seaborn as sns
import pickle
import time
from scipy.signal import butter, filtfilt
import random

In [84]:
pkl_path = '/content/drive/My Drive/Final-Year-Project/Dataset/Final-Version-of-Bird-Classification-Project/feature-extraction/Annotated/Regular/NotAveragePooled/split_features_3s_all_2D.pkl'

# Load the pickle file
with open(pkl_path, 'rb') as file:
    data = pickle.load(file)

In [85]:
train_data = data['train'].copy()
val_data = data['val'].copy()

In [86]:
train_data

{'melspectrogram': array([[[-0.01838665,  0.        , -0.00848863, ..., -0.0140964 ,
          -0.00224623, -0.00832028],
         [-0.1968215 , -0.21882324, -0.27436921, ..., -0.28576213,
          -0.22507286, -0.19316849],
         [-0.23742069, -0.24905342, -0.28791261, ..., -0.28030974,
          -0.26599282, -0.25660488],
         ...,
         [-0.99879366, -0.99876946, -0.99870896, ..., -0.99917048,
          -0.99937928, -0.99937975],
         [-0.9994275 , -0.99943322, -0.99946558, ..., -0.99974817,
          -0.99971521, -0.99969888],
         [-1.        , -1.        , -1.        , ..., -1.        ,
          -1.        , -1.        ]],
 
        [[-0.01671414,  0.        , -0.0082565 , ..., -0.01512463,
          -0.00132613, -0.0053683 ],
         [-0.17772998, -0.19626212, -0.24940659, ..., -0.30918679,
          -0.24149542, -0.20524147],
         [-0.23100212, -0.23447959, -0.25692546, ..., -0.26929191,
          -0.2456764 , -0.23258191],
         ...,
         [-0.99

In [87]:
train_labels = train_data['label'].copy()
temp = train_data.copy()
del temp['label']
tr_features = temp

In [88]:
val_labels = val_data['label'].copy()
temp = val_data.copy()
del temp['label']
v_features = temp

# Shuffle Data

In [89]:
def shuffle_data(input_label, input_features):
  input_len = len(input_label)
  np.random.seed(1826)
  input_indices = np.random.permutation(input_len)
  input_features = {key: np.array([input_features[key][i] for i in input_indices]) for key in input_features} # dictionary comprehension
  input_label = np.array([input_label[i] for i in input_indices])

  return input_label, input_features

In [90]:
train_y, train_features = shuffle_data(train_labels, tr_features)

In [91]:
val_y, val_features = shuffle_data(val_labels, v_features)

In [92]:
display(train_y.shape)
display(train_y[:15])

(5278,)

array([ 1, 16,  1,  2, 13,  3, 16, 15,  1,  0,  9, 13,  2, 17,  5],
      dtype=int32)

In [93]:
for key in train_features.keys():
  display(key)
  display(train_features[key].shape)
  display(train_features[key][0])

'melspectrogram'

(5278, 128, 259)

array([[-0.01707135,  0.        , -0.00803004, ..., -0.01309088,
        -0.00134445, -0.00628396],
       [-0.19095391, -0.20925798, -0.26611498, ..., -0.30659822,
        -0.22971404, -0.1961073 ],
       [-0.26433569, -0.28252903, -0.33811274, ..., -0.37626317,
        -0.30302411, -0.27038145],
       ...,
       [-0.98720688, -0.98624605, -0.98306841, ..., -0.98450774,
        -0.99309105, -0.99470353],
       [-0.99932593, -0.99883336, -0.99588645, ..., -0.99674553,
        -1.        , -1.        ],
       [-1.        , -1.        , -1.        , ..., -1.        ,
        -0.99780911, -0.99684298]])

'mfcc'

(5278, 20, 259)

...

(5278, 84, 259)

array([[-2.26095021e-01, -2.54480004e-01, -4.00108874e-01, ...,
        -2.03836262e-01, -3.83356214e-02, -1.49011612e-08],
       [-4.53062952e-01, -5.30015230e-01, -6.76905394e-01, ...,
        -4.46281791e-01, -2.90680707e-01, -2.05382109e-01],
       [-7.65330911e-01, -7.82241166e-01, -9.86226559e-01, ...,
        -7.43246794e-01, -5.16093910e-01, -4.64929044e-01],
       ...,
       [-1.30036249e+01, -1.90142689e+01, -1.98970470e+01, ...,
        -1.04235544e+01, -1.75874863e+01, -9.66891193e+00],
       [-1.28789825e+01, -1.69870529e+01, -1.28914795e+01, ...,
        -1.12526684e+01, -6.83810806e+00, -1.07536049e+01],
       [-1.55329523e+01, -1.89735603e+01, -1.53286657e+01, ...,
        -1.92332726e+01, -9.36328983e+00, -2.50303764e+01]])

'id'

(5278,)

'XC358628.mp3'

In [94]:
display(val_y.shape)
display(val_y[:15])

(1350,)

array([17,  2,  1,  2, 18,  2,  1,  3,  7,  2,  2, 15,  9, 15,  6],
      dtype=int32)

In [95]:
for key in val_features.keys():
  display(key)
  display(val_features[key].shape)
  display(val_features[key][0])

'melspectrogram'

(1350, 128, 259)

array([[-0.08927982, -0.07154191, -0.07405514, ..., -0.07131123,
        -0.04856302, -0.04848331],
       [-0.25275759, -0.26858278, -0.31829946, ..., -0.30957727,
        -0.2453672 , -0.21418647],
       [-0.32182292, -0.33767052, -0.38655693, ..., -0.3730203 ,
        -0.31960808, -0.29334177],
       ...,
       [-0.99583571, -0.99586676, -0.99619127, ..., -0.99568186,
        -0.99508411, -0.99514051],
       [-0.99720996, -0.99733484, -0.99788771, ..., -0.99575268,
        -0.99455928, -0.99454338],
       [-0.99826414, -0.99861173, -1.        , ..., -1.        ,
        -1.        , -1.        ]])

'mfcc'

(1350, 20, 259)

...

(1350, 84, 259)

array([[ -4.96117946,  -4.98943387,  -5.13477446, ...,  -4.10589573,
         -3.9544641 ,  -3.89248371],
       [ -5.1931474 ,  -5.2702744 ,  -5.41740439, ...,  -6.53222728,
         -5.9276817 ,  -5.43866031],
       [ -5.50660534,  -5.523381  ,  -5.72691386, ...,  -2.27492126,
         -2.23120413,  -2.28983142],
       ...,
       [-24.73274405, -60.34352413, -59.73790802, ..., -51.30912419,
        -56.29529652, -50.66915157],
       [-25.05466204, -59.58331214, -57.62388118, ..., -50.94163721,
        -52.00354656, -44.67068148],
       [-25.22672321, -62.69822785, -63.1191885 , ..., -49.59993179,
        -51.97245995, -53.07590773]])

'id'

(1350,)

'XC252756.mp3'

## **Random Forest Model**

Saving evaluation results

In [96]:
def evaluate_model(model, validation_features, val_y):
    # Predict class labels for validation set
    val_yhat_result = model.predict(validation_features)

    # Print classification report
    print('Validation classification Report \n')
    print(classification_report(val_y, val_yhat_result))

    # Get probabilities for the validation set (for AUC calculation)
    val_y_proba = model.predict_proba(validation_features)

    # Calculate AUC for multiclass classification using 'ovr' and 'weighted' average
    auc_score = roc_auc_score(val_y, val_y_proba, multi_class='ovr', average='weighted')
    print(f'AUC Score: {auc_score}')

    # Calculate F1-score with 'weighted' average for imbalanced dataset
    f1 = f1_score(val_y, val_yhat_result, average='weighted')
    print(f'F1 Score (Weighted): {f1}')

    # Store the scores in the dictionary
    val_score = {'f1': f1, 'auc': auc_score}

    # Return the scores dictionary
    return val_score

In [97]:
train_results = {}
val_results = {}

val_scores = {}
k=15

### **With all the features**

In [98]:
training_features_3D = np.concatenate((train_features['mfcc'], train_features['chroma'], train_features['cqt'], train_features['melspectrogram']), axis=1)
training_features = training_features_3D.reshape(training_features_3D.shape[0], -1)
training_features.shape

(5278, 63196)

In [99]:
validation_features_3D = np.concatenate((val_features['mfcc'], val_features['chroma'], val_features['cqt'], val_features['melspectrogram']), axis=1)
validation_features = validation_features_3D.reshape(validation_features_3D.shape[0], -1)
validation_features.shape

(1350, 63196)

Fit the model with training data

In [100]:
scaler = StandardScaler()
training_features = scaler.fit_transform(training_features)
validation_features = scaler.transform(validation_features)
model = KNeighborsClassifier(n_neighbors=k, weights='distance')
model.fit(training_features, train_y)

KNeighborsClassifier(n_neighbors=15, weights='distance')

In [101]:
print('Training accuracy:', model.score(training_features, train_y))
train_results['all_features'] = model.score(training_features, train_y)

print('Validation accuracy:', model.score(validation_features, val_y))
val_results['all_features'] = model.score(validation_features, val_y)

Training accuracy: 1.0
Validation accuracy: 0.5296296296296297


In [102]:
val_scores['all_features'] = evaluate_model(model=model, validation_features=validation_features, val_y=val_y)

Validation classification Report 

              precision    recall  f1-score   support

           0       0.63      0.19      0.30       114
           1       0.30      0.71      0.42       141
           2       0.62      0.51      0.56       271
           3       0.56      0.89      0.68        90
           4       0.00      0.00      0.00        18
           5       0.43      0.60      0.50        15
           6       0.23      0.39      0.29        23
           7       0.94      1.00      0.97        44
           8       0.37      0.68      0.48        50
           9       0.89      0.51      0.65        49
          10       0.79      0.46      0.58        48
          11       0.69      0.39      0.50        46
          12       0.65      0.98      0.78        54
          13       0.57      0.52      0.54        48
          14       0.30      0.39      0.34        38
          15       0.32      0.09      0.15       117
          16       0.89      0.98      0.93   

### **With MFCCs only**

In [103]:
training_features_3D = train_features['mfcc']
training_features = training_features_3D.reshape(training_features_3D.shape[0], -1)
training_features.shape

(5278, 5180)

In [104]:
validation_features_3D = val_features['mfcc']
validation_features = validation_features_3D.reshape(validation_features_3D.shape[0], -1)
validation_features.shape

(1350, 5180)

In [105]:
scaler = StandardScaler()
training_features = scaler.fit_transform(training_features)
validation_features = scaler.transform(validation_features)
model = KNeighborsClassifier(n_neighbors=k, weights='distance')
model.fit(training_features, train_y)

KNeighborsClassifier(n_neighbors=15, weights='distance')

In [106]:
print('Training accuracy:', model.score(training_features, train_y))
train_results['mfcc'] = model.score(training_features, train_y)

print('Validation accuracy:', model.score(validation_features, val_y))
val_results['mfcc'] = model.score(validation_features, val_y)

Training accuracy: 1.0
Validation accuracy: 0.5414814814814815


In [107]:
val_scores['mfcc'] = evaluate_model(model=model, validation_features=validation_features, val_y=val_y)

Validation classification Report 

              precision    recall  f1-score   support

           0       0.33      0.09      0.14       114
           1       0.22      0.10      0.14       141
           2       0.60      0.81      0.69       271
           3       0.46      1.00      0.63        90
           4       0.00      0.00      0.00        18
           5       0.00      0.00      0.00        15
           6       0.30      0.13      0.18        23
           7       0.56      1.00      0.72        44
           8       0.25      0.86      0.39        50
           9       0.75      0.90      0.81        49
          10       0.83      0.10      0.19        48
          11       0.46      0.35      0.40        46
          12       0.66      0.93      0.77        54
          13       0.69      0.83      0.75        48
          14       0.57      0.76      0.65        38
          15       0.80      0.10      0.18       117
          16       0.76      1.00      0.87   

### **With Mel-Spectrogram Only**

In [108]:
training_features_3D = train_features['melspectrogram']
training_features = training_features_3D.reshape(training_features_3D.shape[0], -1)
training_features.shape

(5278, 33152)

In [109]:
validation_features_3D = val_features['melspectrogram']
validation_features = validation_features_3D.reshape(validation_features_3D.shape[0], -1)
validation_features.shape

(1350, 33152)

In [110]:
scaler = StandardScaler()
training_features = scaler.fit_transform(training_features)
validation_features = scaler.transform(validation_features)
model = KNeighborsClassifier(n_neighbors=k, weights='distance')
model.fit(training_features, train_y)

KNeighborsClassifier(n_neighbors=15, weights='distance')

In [111]:
print('Training accuracy:', model.score(training_features, train_y))
train_results['melspectrogram'] = model.score(training_features, train_y)

print('Validation accuracy:', model.score(validation_features, val_y))
val_results['melspectrogram'] = model.score(validation_features, val_y)

Training accuracy: 1.0
Validation accuracy: 0.5044444444444445


In [112]:
val_scores['melspectrogram'] = evaluate_model(model=model, validation_features=validation_features, val_y=val_y)

Validation classification Report 

              precision    recall  f1-score   support

           0       0.62      0.23      0.33       114
           1       0.25      0.68      0.37       141
           2       0.52      0.32      0.40       271
           3       0.57      0.96      0.71        90
           4       0.20      0.06      0.09        18
           5       0.32      0.60      0.42        15
           6       0.21      0.35      0.26        23
           7       0.87      0.91      0.89        44
           8       0.47      0.72      0.57        50
           9       0.91      0.43      0.58        49
          10       0.64      0.56      0.60        48
          11       0.70      0.35      0.46        46
          12       0.74      0.98      0.84        54
          13       0.66      0.56      0.61        48
          14       0.45      0.63      0.53        38
          15       0.29      0.04      0.07       117
          16       0.85      1.00      0.92   

## **With Chroma Only**

In [113]:
training_features_3D = train_features['chroma']
training_features = training_features_3D.reshape(training_features_3D.shape[0], -1)
training_features.shape

(5278, 3108)

In [114]:
validation_features_3D = val_features['chroma']
validation_features = validation_features_3D.reshape(validation_features_3D.shape[0], -1)
validation_features.shape

(1350, 3108)

In [115]:
scaler = StandardScaler()
training_features = scaler.fit_transform(training_features)
validation_features = scaler.transform(validation_features)
model = KNeighborsClassifier(n_neighbors=k, weights='distance')
model.fit(training_features, train_y)

KNeighborsClassifier(n_neighbors=15, weights='distance')

In [116]:
print('Training accuracy:', model.score(training_features, train_y))
train_results['chroma'] = model.score(training_features, train_y)

print('Validation accuracy:', model.score(validation_features, val_y))
val_results['chroma'] = model.score(validation_features, val_y)

Training accuracy: 1.0
Validation accuracy: 0.3933333333333333


In [117]:
val_scores['chroma'] = evaluate_model(model=model, validation_features=validation_features, val_y=val_y)

Validation classification Report 

              precision    recall  f1-score   support

           0       0.78      0.27      0.40       114
           1       0.28      0.17      0.21       141
           2       0.66      0.39      0.49       271
           3       0.52      0.52      0.52        90
           4       0.00      0.00      0.00        18
           5       0.00      0.00      0.00        15
           6       0.20      0.22      0.21        23
           7       0.87      0.93      0.90        44
           8       0.18      0.08      0.11        50
           9       0.17      0.76      0.28        49
          10       0.88      0.60      0.72        48
          11       0.05      0.04      0.05        46
          12       0.18      0.76      0.29        54
          13       0.30      0.44      0.36        48
          14       0.38      0.71      0.50        38
          15       0.81      0.22      0.35       117
          16       0.46      0.49      0.47   

## **With CQT Only**

In [118]:
training_features_3D = train_features['cqt']
training_features = training_features_3D.reshape(training_features_3D.shape[0], -1)
training_features.shape

(5278, 21756)

In [119]:
validation_features_3D = val_features['cqt']
validation_features = validation_features_3D.reshape(validation_features_3D.shape[0], -1)
validation_features.shape

(1350, 21756)

In [120]:
scaler = StandardScaler()
training_features = scaler.fit_transform(training_features)
validation_features = scaler.transform(validation_features)
model = KNeighborsClassifier(n_neighbors=k, weights='distance')
model.fit(training_features, train_y)

KNeighborsClassifier(n_neighbors=15, weights='distance')

In [121]:
print('Training accuracy:', model.score(training_features, train_y))
train_results['cqt'] = model.score(training_features, train_y)

print('Validation accuracy:', model.score(validation_features, val_y))
val_results['cqt'] = model.score(validation_features, val_y)

Training accuracy: 1.0
Validation accuracy: 0.42518518518518517


In [122]:
val_scores['cqt'] = evaluate_model(model=model, validation_features=validation_features, val_y=val_y)

Validation classification Report 

              precision    recall  f1-score   support

           0       0.47      0.16      0.24       114
           1       0.36      0.66      0.47       141
           2       0.69      0.62      0.66       271
           3       0.14      0.33      0.19        90
           4       0.00      0.00      0.00        18
           5       0.33      0.07      0.11        15
           6       0.35      0.26      0.30        23
           7       0.93      0.93      0.93        44
           8       0.28      0.56      0.37        50
           9       1.00      0.14      0.25        49
          10       0.34      0.40      0.37        48
          11       0.56      0.22      0.31        46
          12       0.76      0.81      0.79        54
          13       0.38      0.33      0.36        48
          14       0.05      0.08      0.06        38
          15       0.02      0.01      0.01       117
          16       0.77      0.85      0.81   

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


AUC Score: 0.8292055056918451
F1 Score (Weighted): 0.40574180307705376


## **With MFCCs and Mel-Spectrogram**

In [123]:
training_features_3D = np.concatenate((train_features['mfcc'], train_features['melspectrogram']), axis=1)
training_features = training_features_3D.reshape(training_features_3D.shape[0], -1)
training_features.shape

(5278, 38332)

In [124]:
validation_features_3D = np.concatenate((val_features['mfcc'], val_features['melspectrogram']), axis=1)
validation_features = validation_features_3D.reshape(validation_features_3D.shape[0], -1)
validation_features.shape

(1350, 38332)

In [125]:
scaler = StandardScaler()
training_features = scaler.fit_transform(training_features)
validation_features = scaler.transform(validation_features)
model = KNeighborsClassifier(n_neighbors=k, weights='distance')
model.fit(training_features, train_y)

KNeighborsClassifier(n_neighbors=15, weights='distance')

In [126]:
print('Training accuracy:', model.score(training_features, train_y))
train_results['mfcc_melspectrogram'] = model.score(training_features, train_y)

print('Validation accuracy:', model.score(validation_features, val_y))
val_results['mfcc_melspectrogram'] = model.score(validation_features, val_y)

Training accuracy: 1.0
Validation accuracy: 0.5185185185185185


In [127]:
val_scores['mfcc_melspectrogram'] = evaluate_model(model=model, validation_features=validation_features, val_y=val_y)

Validation classification Report 

              precision    recall  f1-score   support

           0       0.51      0.20      0.29       114
           1       0.26      0.67      0.38       141
           2       0.53      0.37      0.43       271
           3       0.56      0.97      0.71        90
           4       0.33      0.06      0.10        18
           5       0.40      0.67      0.50        15
           6       0.23      0.35      0.28        23
           7       0.80      0.98      0.88        44
           8       0.51      0.78      0.61        50
           9       0.85      0.45      0.59        49
          10       0.73      0.46      0.56        48
          11       0.70      0.41      0.52        46
          12       0.67      0.98      0.80        54
          13       0.64      0.67      0.65        48
          14       0.56      0.63      0.59        38
          15       0.37      0.09      0.14       117
          16       0.79      1.00      0.88   

## **With MFCCs and Chroma**

In [128]:
training_features_3D = np.concatenate((train_features['mfcc'], train_features['chroma']), axis=1)
training_features = training_features_3D.reshape(training_features_3D.shape[0], -1)
training_features.shape

(5278, 8288)

In [129]:
validation_features_3D = np.concatenate((val_features['mfcc'], val_features['chroma']), axis=1)
validation_features = validation_features_3D.reshape(validation_features_3D.shape[0], -1)
validation_features.shape

(1350, 8288)

In [130]:
scaler = StandardScaler()
training_features = scaler.fit_transform(training_features)
validation_features = scaler.transform(validation_features)
model = KNeighborsClassifier(n_neighbors=k, weights='distance')
model.fit(training_features, train_y)

KNeighborsClassifier(n_neighbors=15, weights='distance')

In [131]:
print('Training accuracy:', model.score(training_features, train_y))
train_results['mfcc_chroma'] = model.score(training_features, train_y)

print('Validation accuracy:', model.score(validation_features, val_y))
val_results['mfcc_chroma'] = model.score(validation_features, val_y)

Training accuracy: 1.0
Validation accuracy: 0.5644444444444444


In [132]:
val_scores['mfcc_chroma'] = evaluate_model(model=model, validation_features=validation_features, val_y=val_y)

Validation classification Report 

              precision    recall  f1-score   support

           0       0.47      0.14      0.22       114
           1       0.37      0.11      0.16       141
           2       0.63      0.81      0.71       271
           3       0.63      0.93      0.75        90
           4       0.00      0.00      0.00        18
           5       0.00      0.00      0.00        15
           6       0.36      0.17      0.24        23
           7       0.64      0.98      0.77        44
           8       0.39      0.80      0.52        50
           9       0.57      0.92      0.70        49
          10       0.95      0.40      0.56        48
          11       0.20      0.54      0.29        46
          12       0.47      0.93      0.62        54
          13       0.67      0.83      0.74        48
          14       0.60      0.79      0.68        38
          15       0.92      0.10      0.18       117
          16       0.87      0.96      0.91   

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


AUC Score: 0.8776779633472203
F1 Score (Weighted): 0.5048555270121403


## **With MFCCs and CQT**

In [133]:
training_features_3D = np.concatenate((train_features['mfcc'], train_features['cqt']), axis=1)
training_features = training_features_3D.reshape(training_features_3D.shape[0], -1)
training_features.shape

(5278, 26936)

In [134]:
validation_features_3D = np.concatenate((val_features['mfcc'], val_features['cqt']), axis=1)
validation_features = validation_features_3D.reshape(validation_features_3D.shape[0], -1)
validation_features.shape

(1350, 26936)

In [135]:
scaler = StandardScaler()
training_features = scaler.fit_transform(training_features)
validation_features = scaler.transform(validation_features)
model = KNeighborsClassifier(n_neighbors=k, weights='distance')
model.fit(training_features, train_y)

KNeighborsClassifier(n_neighbors=15, weights='distance')

In [136]:
print('Training accuracy:', model.score(training_features, train_y))
train_results['mfcc_cqt'] = model.score(training_features, train_y)

print('Validation accuracy:', model.score(validation_features, val_y))
val_results['mfcc_cqt'] = model.score(validation_features, val_y)

Training accuracy: 1.0
Validation accuracy: 0.49703703703703705


In [137]:
val_scores['mfcc_cqt'] = evaluate_model(model=model, validation_features=validation_features, val_y=val_y)

Validation classification Report 

              precision    recall  f1-score   support

           0       0.70      0.27      0.39       114
           1       0.35      0.57      0.43       141
           2       0.64      0.66      0.65       271
           3       0.32      0.74      0.44        90
           4       0.00      0.00      0.00        18
           5       0.33      0.13      0.19        15
           6       0.29      0.26      0.27        23
           7       0.93      0.93      0.93        44
           8       0.30      0.64      0.41        50
           9       1.00      0.20      0.34        49
          10       0.73      0.33      0.46        48
          11       0.74      0.37      0.49        46
          12       0.65      0.85      0.74        54
          13       0.53      0.58      0.55        48
          14       0.15      0.21      0.18        38
          15       0.18      0.08      0.11       117
          16       0.78      0.85      0.82   

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


AUC Score: 0.8480636636876971
F1 Score (Weighted): 0.4753357853535436


## **With Chroma and CQT**

In [138]:
training_features_3D = np.concatenate((train_features['chroma'], train_features['cqt']), axis=1)
training_features = training_features_3D.reshape(training_features_3D.shape[0], -1)
training_features.shape

(5278, 24864)

In [139]:
validation_features_3D = np.concatenate((val_features['chroma'], val_features['cqt']), axis=1)
validation_features = validation_features_3D.reshape(validation_features_3D.shape[0], -1)
validation_features.shape

(1350, 24864)

In [140]:
scaler = StandardScaler()
training_features = scaler.fit_transform(training_features)
validation_features = scaler.transform(validation_features)
model = KNeighborsClassifier(n_neighbors=k, weights='distance')
model.fit(training_features, train_y)

KNeighborsClassifier(n_neighbors=15, weights='distance')

In [141]:
print('Training accuracy:', model.score(training_features, train_y))
train_results['chroma_cqt'] = model.score(training_features, train_y)

print('Validation accuracy:', model.score(validation_features, val_y))
val_results['chroma_cqt'] = model.score(validation_features, val_y)

Training accuracy: 1.0
Validation accuracy: 0.4925925925925926


In [142]:
val_scores['chroma_cqt'] = evaluate_model(model=model, validation_features=validation_features, val_y=val_y)

Validation classification Report 

              precision    recall  f1-score   support

           0       0.59      0.21      0.31       114
           1       0.38      0.70      0.49       141
           2       0.60      0.65      0.63       271
           3       0.38      0.56      0.45        90
           4       0.00      0.00      0.00        18
           5       0.20      0.07      0.10        15
           6       0.25      0.30      0.27        23
           7       0.93      0.95      0.94        44
           8       0.35      0.56      0.43        50
           9       0.54      0.14      0.23        49
          10       0.74      0.58      0.65        48
          11       0.52      0.24      0.33        46
          12       0.73      0.94      0.82        54
          13       0.42      0.42      0.42        48
          14       0.12      0.24      0.16        38
          15       0.22      0.09      0.13       117
          16       0.73      0.85      0.79   

## **With Mel-Spectrogram and Chroma**

In [143]:
training_features_3D = np.concatenate((train_features['melspectrogram'], train_features['chroma']), axis=1)
training_features = training_features_3D.reshape(training_features_3D.shape[0], -1)
training_features.shape

(5278, 36260)

In [144]:
validation_features_3D = np.concatenate((val_features['melspectrogram'], val_features['chroma']), axis=1)
validation_features = validation_features_3D.reshape(validation_features_3D.shape[0], -1)
validation_features.shape

(1350, 36260)

In [145]:
scaler = StandardScaler()
training_features = scaler.fit_transform(training_features)
validation_features = scaler.transform(validation_features)
model = KNeighborsClassifier(n_neighbors=k, weights='distance')
model.fit(training_features, train_y)

KNeighborsClassifier(n_neighbors=15, weights='distance')

In [146]:
print('Training accuracy:', model.score(training_features, train_y))
train_results['melspectrogram_chroma'] = model.score(training_features, train_y)

print('Validation accuracy:', model.score(validation_features, val_y))
val_results['melspectrogram_chroma'] = model.score(validation_features, val_y)

Training accuracy: 1.0
Validation accuracy: 0.5214814814814814


In [147]:
val_scores['melspectrogram_chroma'] = evaluate_model(model=model, validation_features=validation_features, val_y=val_y)

Validation classification Report 

              precision    recall  f1-score   support

           0       0.69      0.19      0.30       114
           1       0.24      0.60      0.35       141
           2       0.51      0.38      0.43       271
           3       0.66      0.96      0.78        90
           4       0.10      0.06      0.07        18
           5       0.42      0.67      0.51        15
           6       0.19      0.35      0.24        23
           7       0.95      0.93      0.94        44
           8       0.49      0.64      0.56        50
           9       0.87      0.53      0.66        49
          10       0.72      0.65      0.68        48
          11       0.63      0.41      0.50        46
          12       0.67      0.98      0.80        54
          13       0.62      0.54      0.58        48
          14       0.42      0.66      0.51        38
          15       0.46      0.10      0.17       117
          16       0.87      1.00      0.93   

## **With Mel-Spectrogram and CQT**

In [148]:
training_features_3D = np.concatenate((train_features['melspectrogram'], train_features['cqt']), axis=1)
training_features = training_features_3D.reshape(training_features_3D.shape[0], -1)
training_features.shape

(5278, 54908)

In [149]:
validation_features_3D = np.concatenate((val_features['melspectrogram'], val_features['cqt']), axis=1)
validation_features = validation_features_3D.reshape(validation_features_3D.shape[0], -1)
validation_features.shape

(1350, 54908)

In [150]:
scaler = StandardScaler()
training_features = scaler.fit_transform(training_features)
validation_features = scaler.transform(validation_features)
model = KNeighborsClassifier(n_neighbors=k, weights='distance')
model.fit(training_features, train_y)

KNeighborsClassifier(n_neighbors=15, weights='distance')

In [151]:
print('Training accuracy:', model.score(training_features, train_y))
train_results['melspectrogram_cqt'] = model.score(training_features, train_y)

print('Validation accuracy:', model.score(validation_features, val_y))
val_results['melspectrogram_cqt'] = model.score(validation_features, val_y)

Training accuracy: 1.0
Validation accuracy: 0.48518518518518516


In [152]:
val_scores['melspectrogram_cqt'] = evaluate_model(model=model, validation_features=validation_features, val_y=val_y)

Validation classification Report 

              precision    recall  f1-score   support

           0       0.57      0.18      0.27       114
           1       0.28      0.70      0.40       141
           2       0.63      0.46      0.53       271
           3       0.41      0.82      0.55        90
           4       0.25      0.06      0.09        18
           5       0.50      0.60      0.55        15
           6       0.21      0.35      0.26        23
           7       0.92      1.00      0.96        44
           8       0.34      0.66      0.45        50
           9       0.88      0.29      0.43        49
          10       0.59      0.35      0.44        48
          11       0.64      0.35      0.45        46
          12       0.75      0.98      0.85        54
          13       0.58      0.46      0.51        48
          14       0.12      0.16      0.14        38
          15       0.16      0.03      0.06       117
          16       0.90      0.96      0.93   

## **With Mel-Spectrogram, MFCCs, CQT**

In [153]:
training_features_3D = np.concatenate((train_features['melspectrogram'], train_features['cqt'], train_features['mfcc']), axis=1)
training_features = training_features_3D.reshape(training_features_3D.shape[0], -1)
training_features.shape

(5278, 60088)

In [154]:
validation_features_3D = np.concatenate((val_features['melspectrogram'], val_features['cqt'], val_features['mfcc']), axis=1)
validation_features = validation_features_3D.reshape(validation_features_3D.shape[0], -1)
validation_features.shape

(1350, 60088)

In [155]:
scaler = StandardScaler()
training_features = scaler.fit_transform(training_features)
validation_features = scaler.transform(validation_features)
model = KNeighborsClassifier(n_neighbors=k, weights='distance')
model.fit(training_features, train_y)

KNeighborsClassifier(n_neighbors=15, weights='distance')

In [156]:
print('Training accuracy:', model.score(training_features, train_y))
train_results['melspectrogram_cqt_mfcc'] = model.score(training_features, train_y)

print('Validation accuracy:', model.score(validation_features, val_y))
val_results['melspectrogram_cqt_mfcc'] = model.score(validation_features, val_y)

Training accuracy: 1.0
Validation accuracy: 0.5148148148148148


In [157]:
val_scores['melspectrogram_cqt_mfcc'] = evaluate_model(model=model, validation_features=validation_features, val_y=val_y)

Validation classification Report 

              precision    recall  f1-score   support

           0       0.65      0.23      0.34       114
           1       0.29      0.70      0.41       141
           2       0.64      0.50      0.56       271
           3       0.45      0.90      0.60        90
           4       0.00      0.00      0.00        18
           5       0.45      0.60      0.51        15
           6       0.22      0.39      0.28        23
           7       0.92      1.00      0.96        44
           8       0.36      0.64      0.46        50
           9       0.89      0.35      0.50        49
          10       0.77      0.35      0.49        48
          11       0.65      0.37      0.47        46
          12       0.66      0.98      0.79        54
          13       0.65      0.58      0.62        48
          14       0.25      0.29      0.27        38
          15       0.29      0.07      0.11       117
          16       0.90      0.98      0.94   

## **With Mel-Spectrogram, MFCCs, Chroma**

In [158]:
training_features_3D = np.concatenate((train_features['melspectrogram'], train_features['chroma'], train_features['mfcc']), axis=1)
training_features = training_features_3D.reshape(training_features_3D.shape[0], -1)
training_features.shape

(5278, 41440)

In [159]:
validation_features_3D = np.concatenate((val_features['melspectrogram'], val_features['chroma'], val_features['mfcc']), axis=1)
validation_features = validation_features_3D.reshape(validation_features_3D.shape[0], -1)
validation_features.shape

(1350, 41440)

In [160]:
scaler = StandardScaler()
training_features = scaler.fit_transform(training_features)
validation_features = scaler.transform(validation_features)
model = KNeighborsClassifier(n_neighbors=k, weights='distance')
model.fit(training_features, train_y)

KNeighborsClassifier(n_neighbors=15, weights='distance')

In [161]:
print('Training accuracy:', model.score(training_features, train_y))
train_results['melspectrogram_chroma_mfcc'] = model.score(training_features, train_y)

print('Validation accuracy:', model.score(validation_features, val_y))
val_results['melspectrogram_chroma_mfcc'] = model.score(validation_features, val_y)

Training accuracy: 1.0
Validation accuracy: 0.5333333333333333


In [162]:
val_scores['melspectrogram_cqt_mfcc'] = evaluate_model(model=model, validation_features=validation_features, val_y=val_y)

Validation classification Report 

              precision    recall  f1-score   support

           0       0.62      0.20      0.30       114
           1       0.26      0.60      0.36       141
           2       0.50      0.41      0.45       271
           3       0.64      0.98      0.77        90
           4       0.00      0.00      0.00        18
           5       0.38      0.67      0.49        15
           6       0.24      0.39      0.30        23
           7       0.93      0.98      0.96        44
           8       0.50      0.74      0.60        50
           9       0.88      0.57      0.69        49
          10       0.79      0.54      0.64        48
          11       0.73      0.41      0.53        46
          12       0.63      0.98      0.77        54
          13       0.66      0.65      0.65        48
          14       0.54      0.68      0.60        38
          15       0.38      0.11      0.17       117
          16       0.79      1.00      0.88   

# Review results from all models

In [163]:
train_results_df = pd.DataFrame(list(train_results.items()), columns=['Features', 'Train_Accuracy']).round(3)
val_results_df = pd.DataFrame(list(val_results.items()), columns=['Features', 'Val_Accuracy']).round(3)

result_df = train_results_df.merge(val_results_df, on='Features')
result_df = result_df.sort_values('Features')
result_df

,Features,Train_Accuracy,Val_Accuracy
0,all_features,1.0,0.530
3,chroma,1.0,0.393
8,chroma_cqt,1.0,0.493
4,cqt,1.0,0.425
2,melspectrogram,1.0,0.504
9,melspectrogram_chroma,1.0,0.521
12,melspectrogram_chroma_mfcc,1.0,0.533
10,melspectrogram_cqt,1.0,0.485
11,melspectrogram_cqt_mfcc,1.0,0.515
1,mfcc,1.0,0.541


In [164]:
val_scores_df = pd.DataFrame([(key, value['f1'], value['auc']) for key, value in val_scores.items()],
                             columns=['Features', 'F1_Score', 'AUC_Score']).round(3)

val_scores_df = val_scores_df.sort_values('Features')
print(val_scores_df)

                   Features  F1_Score  AUC_Score
0              all_features     0.510      0.880
3                    chroma     0.402      0.832
8                chroma_cqt     0.469      0.862
4                       cqt     0.406      0.829
2            melspectrogram     0.484      0.869
9     melspectrogram_chroma     0.509      0.877
10       melspectrogram_cqt     0.466      0.861
11  melspectrogram_cqt_mfcc     0.515      0.879
1                      mfcc     0.472      0.863
6               mfcc_chroma     0.505      0.878
7                  mfcc_cqt     0.475      0.848
5       mfcc_melspectrogram     0.495      0.877
